In [1]:
# Based on the figures that we got, we found a very interesting point. Most of the  'annual production'   
# will experience a slight increase at the beginning.

# So after the discussion , we think that the main reason is that the data in the first month is incomplete.  
# In other words, instead of installing and recording the data at the first day in a month, most panels may be 
# installed in the middle or some other days in a month, which make the data for this month is not full.

# And this , resulting the annual produciton including the first month will slight less than the next 
# annual production which including the same month in the next year.

# SO WE DECIDE TO DROP THE FIRST MONTH IN OUR DATAFRAME!

In [2]:
import numpy as np
import pandas as pd
#import csv_figures
#from csv_figures import add_time_col
#from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [3]:
def annual_Norm(dataframe):
    """This is a function that calculateing the annual production of the panel
       
       Args:
           dataframe(string):The name of the dataframe you want to adjust
       
       Returns:
           annual_values(list): the sum of the past 12 months energy production
           month(list):the month corresponding to the annual_values
         
    """
    # create three list
    lenth_list = list(range(12,len(dataframe.index)))
    annual_values = []
    month = []
    for i in range(len(lenth_list)):
        single_values = dataframe['Energy'][lenth_list[i]-12:lenth_list[i]].sum()/dataframe['DC Capacity'][0]
        #rolling_average.append(each_period)
        single_month = dataframe['Date'][lenth_list[i]]
        # append the result we want to the list
        annual_values.append(single_values)
        month.append(single_month)
    month.pop(0) # Here we drop the first annual production!
    annual_values.pop(0)# Here we drop the first annual production!
    
    return(annual_values,month)

In [4]:
# ALL right, right now let's see if this can work.

In [5]:

def add_time_col(dataframe):
    """This function can extract the date in the specific format of the dataframe 
       and then create a new column in the dataframe showing the date in the form of the the datetime
       
       Args:
           dataframe(string):The name of the dataframe you want to adjust
       
       Returns:
           A new dataframe with the same name.
    """
    
    # Creating a dictionary that we can use to replace the string to int
    month_change = {'November':11,'December':12,'January': 1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10}
    
    # Check if the input is a dataframe, if not,it won't give the right
    #assert dataframe == pd.DataFrame,"The input should be a dataframe"
    
    # if the input is a dataframe, then we can adjust it
    dataframe['Month'] = dataframe['Month'].map(month_change)
    dataframe['Date'] = ""
    for i in range(len(dataframe)):
        dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])
    
    # Change the date into a datetime format
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    # Since we have a column that shows the 'orderable' format in date, then we can drop the original one
    dataframe.drop(['Year','Month'],axis = 1,inplace = True)
    order = ['Date','Energy','DC Capacity','Location']
    dataframe_new = dataframe[order]
    
    return(dataframe_new)

In [6]:
DBoccia = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16DBoccia_cleaned.xlsx')
LWaller = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16LWaller_cleaned.xlsx')
MAngstman = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16Mangstman_cleaned.xlsx')
May = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16May_cleaned.xlsx')
Pcrofut = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16Pcrofut_cleaned.xlsx')
SDowns = pd.read_excel('../Cleaned_data/AnchorageSolar_cleaned/16SDowns_cleaned.xlsx')

DBoccia_new = add_time_col(DBoccia)
LWaller_new = add_time_col(LWaller)
MAngstman_new = add_time_col(MAngstman)
May_new = add_time_col(May)
Pcrofut_new = add_time_col(Pcrofut)
SDowns_new = add_time_col(SDowns)

D_annual_values,D_month = annual_Norm(DBoccia_new)
L_annual_values,L_month = annual_Norm(LWaller_new)
MA_annual_values,MA_month = annual_Norm(MAngstman_new)
May_annual_values,May_month = annual_Norm(May_new)
P_annual_values,P_month = annual_Norm(Pcrofut_new)
S_annual_values,S_month = annual_Norm(SDowns_new)

new_D = pd.DataFrame({'Date':D_month,'DBoccia':D_annual_values})
new_L = pd.DataFrame({'Date':L_month,'LWaller':L_annual_values})
new_MA = pd.DataFrame({'Date':MA_month,'MAngstman':MA_annual_values})
new_May = pd.DataFrame({'Date':May_month,'May':May_annual_values})
new_P = pd.DataFrame({'Date':P_month,'Pcrofut':P_annual_values})
new_S = pd.DataFrame({'Date':S_month,'SDowns':S_annual_values})
result = pd.merge(new_D,new_L,on='Date',how='outer')
result = pd.merge(result,new_MA,on='Date',how='outer')
result = pd.merge(result,new_May,on='Date',how='outer')
result = pd.merge(result,new_P,on='Date',how='outer')
Anchorage_16= pd.merge(result,new_S,on='Date',how='outer')
# #Anchorage_16 = result.fillna("")
Anchorage_16['Mean'] = Anchorage_16.mean(axis=1)
Anchorage_16 = Anchorage_16.sort_values(by = ['Date'])
Anchorage_16

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,DBoccia,LWaller,MAngstman,May,Pcrofut,SDowns,Mean
0,2017-08-01,461.732143,NaN,NaN,478.543003,NaN,NaN,470.137573
1,2017-09-01,495.263393,NaN,NaN,475.326787,792.156205,NaN,587.582128
2,2017-10-01,512.937500,919.548984,1069.542857,468.768228,810.932991,NaN,756.346112
3,2017-11-01,519.319196,904.084141,1043.536735,462.867327,814.098170,NaN,748.781114
4,2017-12-01,516.753884,896.636953,1037.812245,458.912372,807.618705,2558.336680,1046.011807
5,2018-01-01,517.325848,903.751758,1034.922449,460.173784,807.694152,2562.625547,1047.748923
6,2018-02-01,517.497812,901.046992,1050.767347,460.669339,806.897187,2559.288437,1049.361186
7,2018-03-01,525.674152,891.812617,1027.036735,457.203874,805.841920,2525.520586,1038.848314
8,2018-04-01,546.946473,842.293086,926.069388,438.570180,800.677857,2465.067461,1003.270741
9,2018-05-01,579.252277,825.316523,914.555102,428.858468,804.718036,2443.579180,999.379931


In [9]:
# So now, let's try to draw a plot with checkbox
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = Anchorage_16['Date']


y_D = Anchorage_16['DBoccia']
l0, = ax.plot(x ,y_D,label='DBoccia')

y_L = Anchorage_16['LWaller']
l1, = ax.plot(x ,y_L,label='LWaller')


y_M = Anchorage_16['MAngstman']
l2, = ax.plot(x ,y_M)


y_May = Anchorage_16['May']
l3, = ax.plot(x ,y_May)

y_P = Anchorage_16['Pcrofut']
l4, = ax.plot(x,y_P,label='Pcrofut')


#y_S = Anchorage_16['SDowns']
#l5, = ax.plot(x,y_S,label = 'SDowns(BP)')

y_mean = Anchorage_16['Mean']
l6, = ax.plot(x,y_mean,label='Mean')

#y_TMY2 = Anchorage_16['TMY2']
#l7, = ax.plot(x,y_TMY2,'--',label = 'TMY2_ac_monthly')

#y_TMY3 = Anchorage_16['TMY3']
#l8, = ax.plot(x,y_TMY3,'--',label = 'TMY3_ac_monthly')

#y_TMY2_low = Anchorage_16['TMY2_low']
#l9, = ax.plot(x,y_TMY2_low,'--',label = 'TMY2_low')

#y_TMY2_high = Anchorage_16['TMY2_high']
#l10, = ax.plot(x,y_TMY2_high,'--',label = 'TMY2_high')

#y_TMY3_low = Anchorage_16['TMY3_low']
#l11, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

#y_TMY3_high = Anchorage_16['TMY3_high']
#l12, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')

# # #ax.plot(x=range(len(result['Date'][49:90])),y = result['Mean'][49:90],label='Average')
# # #ax.plot(x=range(len(result['Date'][49:90])),y = result['TMY2_low'][49:90],label='TMY2_low') 
# #ax.plot(x=range(len(result['Date'][49:90])),y = result['TMY2_high'][49:90],label ='TMY2_high')

#this is where we can adjust the subplots side/top/buttom width
# tick_spacing = np.linspace(1, len(Anchorage_16.index)-1,12, dtype = 'int')
# Date_shown = []
# for i in tick_spacing:
#     Date_shown.append(Anchorage_16['Date'][i])
#ax.xticks(tick_spacing,Date_shown)
#plt.annotate('THY2_high',xy=(30,1072.991757),xytext=(32,1100),arrowprops=dict(facecolor='black', shrink=0.05))
#plt.annotate('THY2_low',xy=(10,970.802066),xytext=(6,930),arrowprops=dict(facecolor='black', shrink=0.05))
# ax.set_xticks(tick_spacing)
ax.set_title("Rolling 12-Month Average of Produced Power, Over Time", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("AC Power Produced (kWh) per DC Power Installed (kW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2, l3, l4, l6]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.5, 0.15, 0.4])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# Some 

plt.show()

#Ok,this seems to be fine,maybe we will talk about it after.